# Tarea 1

## Alumno: Gerardo de Miguel González

### Generalización (capas ocultas)

```r
lin<-read.csv('circle.csv',header= F)
a<-as.matrix(lin[,-3])
b<-as.matrix(lin[,3])
backprop_mlp(a,b, h=5, epochs= 500, eta = 0.1)  
# h: hidden neurons
```

Generalizar la función `backprop_mlp` anterior para que contemple la inclusión de una capa oculta. Aplicar la función al ejemplo de la clasificación circular. 

Si se fija el número máximo de épocas en 1000. ¿Qué número de neuronas ocultas y qué valor de la tasa de aprendizaje (eta) es óptimo para este problema? (basta una solución aproximada).

### BONUS 1 ¿Sabrías incluir un término de inercia en el método de backpropagation?

NOTA: ver transparencia 4

### BONUS 2 Generalizar a un número arbitrario de capas

Recomendación: definir los pesos como una lista de matrices, una para cada capa